# LlamaIndex re-ranker and Elasticsearch re-ranker: Comparison review

This notebook demonstrates how to use AutoGen with Elasticsearch. This notebook is based on the article [LlamaIndex re-ranker and Elasticsearch re-ranker: Comparison review](https://www.elastic.co/search-labs/blog/llamaIndex-reranker-and-elasticsearch-reranker-comparison-review).

## Dataset

In [1]:
products = [
    {
        "name": "ASUS ROG Strix G16",
        "description": "Powerful gaming laptop with Intel Core i9 and RTX 4070.",
        "price": 1899.99,
        "reviews": 4.7,
        "sales": 320,
        "features": [
            "Intel Core i9",
            "RTX 4070",
            "16GB RAM",
            "512GB SSD",
            "165Hz Display",
        ],
    },
    {
        "name": "Razer Blade 15",
        "description": "Premium gaming laptop with an ultra-slim design and high refresh rate.",
        "price": 2499.99,
        "reviews": 4.6,
        "sales": 290,
        "features": [
            "Intel Core i7",
            "RTX 4060",
            "16GB RAM",
            "1TB SSD",
            "240Hz Display",
        ],
    },
    {
        "name": "Acer Predator Helios 300",
        "description": "Affordable yet powerful gaming laptop with RTX graphics.",
        "price": 1399.99,
        "reviews": 4.5,
        "sales": 500,
        "features": [
            "Intel Core i7",
            "RTX 3060",
            "16GB RAM",
            "512GB SSD",
            "144Hz Display",
        ],
    },
    {
        "name": "MSI Stealth 17",
        "description": "High-performance gaming laptop with a 17-inch display.",
        "price": 2799.99,
        "reviews": 4.8,
        "sales": 200,
        "features": ["Intel Core i9", "RTX 4080", "32GB RAM", "1TB SSD", "4K Display"],
    },
    {
        "name": "Dell XPS 15",
        "description": "Sleek and powerful ultrabook with a high-resolution display.",
        "price": 2199.99,
        "reviews": 4.7,
        "sales": 350,
        "features": [
            "Intel Core i7",
            "RTX 3050 Ti",
            "16GB RAM",
            "1TB SSD",
            "OLED Display",
        ],
    },
    {
        "name": "HP Omen 16",
        "description": "Gaming laptop with a balanced price-to-performance ratio.",
        "price": 1599.99,
        "reviews": 4.4,
        "sales": 280,
        "features": [
            "AMD Ryzen 7",
            "RTX 3060",
            "16GB RAM",
            "512GB SSD",
            "165Hz Display",
        ],
    },
    {
        "name": "Lenovo Legion 5 Pro",
        "description": "Powerful Ryzen-powered gaming laptop with high refresh rate.",
        "price": 1799.99,
        "reviews": 4.6,
        "sales": 400,
        "features": [
            "AMD Ryzen 9",
            "RTX 3070 Ti",
            "16GB RAM",
            "1TB SSD",
            "165Hz Display",
        ],
    },
    {
        "name": "MacBook Pro 16",
        "description": "Apple's most powerful laptop with M3 Max chip.",
        "price": 3499.99,
        "reviews": 4.9,
        "sales": 500,
        "features": [
            "Apple M3 Max",
            "32GB RAM",
            "1TB SSD",
            "Liquid Retina XDR Display",
        ],
    },
    {
        "name": "Alienware m18",
        "description": "High-end gaming laptop with extreme performance.",
        "price": 2999.99,
        "reviews": 4.8,
        "sales": 150,
        "features": [
            "Intel Core i9",
            "RTX 4090",
            "32GB RAM",
            "2TB SSD",
            "480Hz Display",
        ],
    },
    {
        "name": "Samsung Galaxy Book3 Ultra",
        "description": "Ultra-lightweight yet powerful laptop with AMOLED display.",
        "price": 2099.99,
        "reviews": 4.5,
        "sales": 180,
        "features": [
            "Intel Core i7",
            "RTX 4070",
            "16GB RAM",
            "512GB SSD",
            "AMOLED Display",
        ],
    },
    {
        "name": "Microsoft Surface Laptop 5",
        "description": "Sleek productivity laptop with great battery life.",
        "price": 1699.99,
        "reviews": 4.3,
        "sales": 220,
        "features": ["Intel Core i7", "16GB RAM", "512GB SSD", "Touchscreen"],
    },
    {
        "name": "Gigabyte AORUS 17",
        "description": "Performance-focused gaming laptop with powerful cooling.",
        "price": 1999.99,
        "reviews": 4.6,
        "sales": 250,
        "features": [
            "Intel Core i9",
            "RTX 4070",
            "16GB RAM",
            "1TB SSD",
            "360Hz Display",
        ],
    },
]

## User question

In [65]:
user_query = "Best laptops for gaming"

## LlamaIndex Reranking

### Installing dependencies and import packages

In [3]:
%pip install llama-index-core llama-index-llms-openai rank-llm llama-index-postprocessor-rankgpt-rerank llama-index-embeddings-openai


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import nest_asyncio
from getpass import getpass

from llama_index.core import Document, VectorStoreIndex, QueryBundle, Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank
from llama_index.llms.openai import OpenAI


nest_asyncio.apply()

### Setup keys

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI Key: ")

### Indexing data to llamaIndex

In [6]:
document_objects = []

for doc in products:
    text_content = f"""
      Product Name: {doc["name"]}
      Description: {doc["description"]}
      Price: ${doc["price"]}
      Reviews: {doc["reviews"]} stars
      Sales: {doc["sales"]} units sold
      Features: {', '.join(doc["features"])}
      """

    document_objects.append(Document(text=text_content))

index = VectorStoreIndex.from_documents(document_objects)

### LLM setup

In [7]:
Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

### LlamaIndex rerank feature

In [8]:
# Re-rank method
def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=5, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = RankGPTRerank(
            llm=OpenAI(
                model="gpt-3.5-turbo-16k",
                temperature=0.0,
                api_key=os.environ["OPENAI_API_KEY"],
            ),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)

    return retrieved_nodes

In [9]:
# Function to format the resulting documents.
def visualize_retrieved_nodes(nodes):
    formatted_results = []

    for node in nodes:
        text = node.node.get_text()

        product_name = text.split("Product Name:")[1].split("\n")[0].strip()
        price = text.split("Price:")[1].split("\n")[0].strip()
        reviews = text.split("Reviews:")[1].split("\n")[0].strip()
        features = text.split("Features:")[1].strip()

        formatted_result = f"{price} - {product_name} ({reviews}) {features}"
        formatted_results.append(formatted_result)

    return formatted_results

### Without rerank

In [72]:
new_nodes = get_retrieved_nodes(
    user_query,
    vector_top_k=5,
    with_reranker=False,
)

results = visualize_retrieved_nodes(new_nodes)

print("\nTop 5 results without rerank:")
for idx, result in enumerate(results, start=1):
    print(f"{idx}. {result}")


Top 5 results without rerank:
1. $2499.99 - Razer Blade 15 (4.6 stars) Intel Core i7, RTX 4060, 16GB RAM, 1TB SSD, 240Hz Display
2. $1899.99 - ASUS ROG Strix G16 (4.7 stars) Intel Core i9, RTX 4070, 16GB RAM, 512GB SSD, 165Hz Display
3. $1999.99 - Gigabyte AORUS 17 (4.6 stars) Intel Core i9, RTX 4070, 16GB RAM, 1TB SSD, 360Hz Display
4. $2799.99 - MSI Stealth 17 (4.8 stars) Intel Core i9, RTX 4080, 32GB RAM, 1TB SSD, 4K Display
5. $2999.99 - Alienware m18 (4.8 stars) Intel Core i9, RTX 4090, 32GB RAM, 2TB SSD, 480Hz Display


### With rerank

In [71]:
new_nodes = get_retrieved_nodes(
    user_query,
    vector_top_k=5,
    reranker_top_n=5,
    with_reranker=True,
)

results = visualize_retrieved_nodes(new_nodes)

print("\nTop 5 results with reranking:")
for idx, result in enumerate(results, start=1):
    print(f"{idx}. {result}")

After Reranking, new rank list for nodes: [1, 4, 3, 2, 0]
Top 5 results with reranking:
1. $1899.99 - ASUS ROG Strix G16 (4.7 stars) Intel Core i9, RTX 4070, 16GB RAM, 512GB SSD, 165Hz Display
2. $2999.99 - Alienware m18 (4.8 stars) Intel Core i9, RTX 4090, 32GB RAM, 2TB SSD, 480Hz Display
3. $2799.99 - MSI Stealth 17 (4.8 stars) Intel Core i9, RTX 4080, 32GB RAM, 1TB SSD, 4K Display
4. $1999.99 - Gigabyte AORUS 17 (4.6 stars) Intel Core i9, RTX 4070, 16GB RAM, 1TB SSD, 360Hz Display
5. $2499.99 - Razer Blade 15 (4.6 stars) Intel Core i7, RTX 4060, 16GB RAM, 1TB SSD, 240Hz Display


## Elasticsearch Reranking

### Installing dependencies and import packages

In [25]:
%pip install elasticsearch==8.17


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

### Setup keys

In [ ]:
os.environ["ELASTICSEARCH_ENDPOINT"] = getpass("Elastic Endpoint: ")
os.environ["ELASTICSEARCH_API_KEY"] = getpass("Elastic Api Key: ")

INDEX_NAME = "products-laptops"
INFERENCE_RERANK_NAME = "my-elastic-rerank"

### Elasticsearch client

In [49]:
_client = Elasticsearch(
    os.environ["ELASTICSEARCH_ENDPOINT"],
    api_key=os.environ["ELASTICSEARCH_API_KEY"],
)

### Mappings

In [50]:
try:
    _client.indices.create(
        index=INDEX_NAME,
        body={
            "mappings": {
                "properties": {
                    "name": {"type": "text", "copy_to": "semantic_field"},
                    "description": {"type": "text", "copy_to": "semantic_field"},
                    "price": {"type": "float", "copy_to": "semantic_field"},
                    "reviews": {"type": "float", "copy_to": "semantic_field"},
                    "sales": {"type": "integer", "copy_to": "semantic_field"},
                    "features": {"type": "keyword", "copy_to": "semantic_field"},
                    "semantic_field": {"type": "semantic_text"},
                }
            }
        },
    )

    print("index created successfully")
except Exception as e:
    print(
        f"Error creating inference endpoint: {e.info['error']['root_cause'][0]['reason'] }"
    )

index created successfully


### Inference rerank endpoint

In [51]:
try:
    _client.options(
        request_timeout=60, max_retries=3, retry_on_timeout=True
    ).inference.put(
        task_type="rerank",
        inference_id=INFERENCE_RERANK_NAME,
        body={
            "service": "elasticsearch",
            "service_settings": {
                "model_id": ".rerank-v1",
                "num_threads": 1,
                "adaptive_allocations": {
                    "enabled": True,
                    "min_number_of_allocations": 1,
                    "max_number_of_allocations": 4,
                },
            },
        },
    )

    print("Inference endpoint created successfully.")

except Exception as e:
    print(
        f"Error creating inference endpoint: {e.info['error']['root_cause'][0]['reason'] }"
    )

Inference endpoint created successfully.


### Ingesting documents to Elasticsearch

In [52]:
def build_data():
    for doc in products:
        yield {"_index": INDEX_NAME, "_source": doc}


try:
    success, errors = bulk(_client, build_data())
    print(f"{success} documents indexed successfully")

    if errors:
        print("Errors during indexing:", errors)

except Exception as e:
    print(f"Error: {str(e)}, please wait some seconds and try again.")

12 documents indexed successfully


In [53]:
# Function to execute search queries
async def es_search(query):
    response = _client.search(index=INDEX_NAME, body=query)
    hits = response["hits"]["hits"]

    if not hits:
        return ""

    return hits

In [54]:
# Function to format the results
def format_es_results(hits):
    formatted_results = []

    for hit in hits:
        source = hit["_source"]
        name = source.get("name")
        price = source.get("price")
        reviews = source.get("reviews")
        features = source.get("features")

        formatted_result = f"{price} - {name} ({reviews}) {features}"
        formatted_results.append(formatted_result)

    return formatted_results

### Semantic query

In [68]:
semantic_results = await es_search(
    {
        "size": 5,
        "query": {
            "semantic": {
                "field": "semantic_field",
                "query": user_query,
            }
        },
        "_source": {
            "includes": [
                "name",
                "description",
                "price",
                "reviews",
                "sales",
                "features",
            ]
        },
    }
)

semantic_formatted_results = format_es_results(semantic_results)

print("Query results:")
for idx, result in enumerate(semantic_formatted_results, start=1):
    print(f"{idx}. {result}")

Query results:
1. 2999.99 - Alienware m18 (4.8) ['Intel Core i9', 'RTX 4090', '32GB RAM', '2TB SSD', '480Hz Display']
2. 2799.99 - MSI Stealth 17 (4.8) ['Intel Core i9', 'RTX 4080', '32GB RAM', '1TB SSD', '4K Display']
3. 1599.99 - HP Omen 16 (4.4) ['AMD Ryzen 7', 'RTX 3060', '16GB RAM', '512GB SSD', '165Hz Display']
4. 1399.99 - Acer Predator Helios 300 (4.5) ['Intel Core i7', 'RTX 3060', '16GB RAM', '512GB SSD', '144Hz Display']
5. 1999.99 - Gigabyte AORUS 17 (4.6) ['Intel Core i9', 'RTX 4070', '16GB RAM', '1TB SSD', '360Hz Display']


In [70]:
rerank_results = await es_search(
    {
        "size": 5,
        "_source": {
            "includes": [
                "name",
                "description",
                "price",
                "reviews",
                "sales",
                "features",
            ]
        },
        "retriever": {
            "text_similarity_reranker": {
                "retriever": {
                    "standard": {
                        "query": {
                            "semantic": {
                                "field": "semantic_field",
                                "query": user_query,
                            }
                        }
                    }
                },
                "field": "semantic_field",
                "inference_id": INFERENCE_RERANK_NAME,
                "inference_text": "reorder by quality-price ratio",
                "rank_window_size": 5,
            }
        },
    }
)

rerank_formatted_results = format_es_results(rerank_results)

print("Query results:")
for idx, result in enumerate(rerank_formatted_results, start=1):
    print(f"{idx}. {result}")

Query results:
1. 1399.99 - Acer Predator Helios 300 (4.5) ['Intel Core i7', 'RTX 3060', '16GB RAM', '512GB SSD', '144Hz Display']
2. 2999.99 - Alienware m18 (4.8) ['Intel Core i9', 'RTX 4090', '32GB RAM', '2TB SSD', '480Hz Display']
3. 2799.99 - MSI Stealth 17 (4.8) ['Intel Core i9', 'RTX 4080', '32GB RAM', '1TB SSD', '4K Display']
4. 1999.99 - Gigabyte AORUS 17 (4.6) ['Intel Core i9', 'RTX 4070', '16GB RAM', '1TB SSD', '360Hz Display']
5. 1599.99 - HP Omen 16 (4.4) ['AMD Ryzen 7', 'RTX 3060', '16GB RAM', '512GB SSD', '165Hz Display']


## Cleaning environment

Delete the resources used to prevent them from consuming resources.

In [46]:
def print_results(results):
    if results.get("acknowledged", False):
        print("DELETED successfully.")

    if "error" in results:
        print(f"ERROR: {results['error']['root_cause'][0]['reason']}")


# Cleanup - Delete Index
result = _client.indices.delete(index=INDEX_NAME, ignore=[400, 404])
print_results(result)


# Cleanup - Inference endpoint
result = _client.inference.delete(inference_id=INFERENCE_RERANK_NAME, ignore=[400, 404])
print_results(result)

/var/folders/5w/qq_73hhn37v6k0qn0gqscwsw0000gn/T/ipykernel_22470/2652817320.py:10: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = _client.indices.delete(index=INDEX_NAME, ignore=[400, 404])


DELETED successfully.


/var/folders/5w/qq_73hhn37v6k0qn0gqscwsw0000gn/T/ipykernel_22470/2652817320.py:15: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = _client.inference.delete(inference_id=INFERENCE_RERANK_NAME, ignore=[400, 404])


DELETED successfully.
